In [1]:
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point
import random
import ast

# prendre une cartede plus haute resolution pour plus de pays
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) 
climate_raster = 'map.tif'


In [2]:
import geopandas as gpd

# Path to your shapefile
shapefile_path = 'map/c1976_2000.shp'

# Load the shapefile using GeoPandas
climate_shapefile = gpd.read_file(shapefile_path)

# Step 1: Display the column names to understand what data is available
print("Column Names in the Shapefile:")
print(climate_shapefile.columns)

# Step 2: Check the first few rows to inspect the structure of the data
print("\nFirst few rows of the Shapefile:")
print(climate_shapefile.head())

# Step 3: Check the data types of each column
print("\nData types of each column:")
print(climate_shapefile.dtypes)

# Step 4: (Optional) Describe the numeric columns, if any, to get an overview of the data
print("\nSummary of numeric columns:")
print(climate_shapefile.describe())

# Step 5: (Optional) If you expect a geometry column, check the geometry of the shapefile
print("\nGeometry column (geometries present in the shapefile):")
print(climate_shapefile.geometry)


In [3]:
import geopandas as gpd

# Path to your shapefile
shapefile_path = 'map/c1976_2000.shp'

# Load the shapefile using GeoPandas
climate_shapefile = gpd.read_file(shapefile_path)

# Display the first few rows to understand the structure
print(climate_shapefile.head())

# Since there's no 'country' column, you need to intersect the geometries with a world borders dataset
# You can use a world borders dataset provided by GeoPandas, like "naturalearth_lowres"
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Ensure both datasets use the same coordinate reference system (CRS)
# Reproject the world data to match the shapefile CRS if needed
if climate_shapefile.crs != world.crs:
    world = world.to_crs(climate_shapefile.crs)

# Perform a spatial join to map climate polygons to countries
# This will give each climate polygon a corresponding country based on spatial intersection
climate_with_countries = gpd.sjoin(climate_shapefile, world, how="inner", op="intersects")

# Now group the data by country and get the unique climate classifications (GRIDCODEs) for each country
climate_per_country = climate_with_countries.groupby('name')['GRIDCODE'].unique()

# Convert to a dictionary for easier access
climate_per_country_dict = climate_per_country.to_dict()

# Display the climate types for each country
for country, climates in climate_per_country_dict.items():
    print(f"Country: {country}, Climate Types: {climates}")


In [4]:
 df =pd.DataFrame([climate_per_country_dict])

In [5]:
df.transpose().to_csv('climate_names.csv')

In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv('climate_names.csv')
df.rename(columns={'Unnamed: 0': 'country'}, inplace=True)
df.rename(columns={'0': 'climate'}, inplace=True)


def climate_to_continuous(koppen_code):
    temperature_map = {
        11: 1.0,  # Af
        12: 0.9,  # Am
        13: 0.8,  # As
        14: 0.7,  # Aw
        21: 0.5,  # BWk
        22: 0.6,  # BWh
        26: 0.3,  # BSk
        27: 0.4,  # BSh
        31: 0.6,  # Cfa
        32: 0.5,  # Cfb
        33: 0.4,  # Cfc
        34: 0.7,  # Csa
        35: 0.5,  # Csb
        36: 0.4,  # Csc
        37: 0.5,  # Cwa
        38: 0.4,  # Cwb
        39: 0.3,  # Cwc
        41: 0.6,  # Dfa
        42: 0.5,  # Dfb
        43: 0.4,  # Dfc
        44: 0.3,  # Dfd
        45: 0.4,  # Dsa
        46: 0.4,  # Dsb
        47: 0.4,  # Dsc
        48: 0.4,  # Dsd
        49: 0.3,  # Dwa
        50: 0.2,  # Dwb
        51: 0.1,  # Dwc
        52: 0.1,  # Dwd
        61: 0.1,  # EF
        62: 0.1   # ET
    }
    
    precipitation_map = {
        11: 1.0,  # Af
        12: 0.9,  # Am
        13: 0.9,  # As
        14: 0.7,  # Aw
        21: 0.1,  # BWk
        22: 0.2,  # BWh
        26: 0.3,  # BSk
        27: 0.4,  # BSh
        31: 0.7,  # Cfa
        32: 0.6,  # Cfb
        33: 0.5,  # Cfc
        34: 0.8,  # Csa
        35: 0.6,  # Csb
        36: 0.5,  # Csc
        37: 0.7,  # Cwa
        38: 0.8,  # Cwb
        39: 0.6,  # Cwc
        41: 0.5,  # Dfa
        42: 0.4,  # Dfb
        43: 0.3,  # Dfc
        44: 0.2,  # Dfd
        45: 0.4,  # Dsa
        46: 0.4,  # Dsb
        47: 0.4,  # Dsc
        48: 0.4,  # Dsd
        49: 0.3,  # Dwa
        50: 0.2,  # Dwb
        51: 0.1,  # Dwc
        52: 0.1,  # Dwd
        61: 0.1,  # EF
        62: 0.1   # ET
    }
    
    seasonality_map = {
        11: 1.0,  # Af
        12: 0.8,  # Am
        13: 0.7,  # As
        14: 0.6,  # Aw
        21: 0.1,  # BWk
        22: 0.1,  # BWh
        26: 0.2,  # BSk
        27: 0.3,  # BSh
        31: 0.6,  # Cfa
        32: 0.5,  # Cfb
        33: 0.4,  # Cfc
        34: 0.5,  # Csa
        35: 0.5,  # Csb
        36: 0.4,  # Csc
        37: 0.5,  # Cwa
        38: 0.4,  # Cwb
        39: 0.3,  # Cwc
        41: 0.5,  # Dfa
        42: 0.4,  # Dfb
        43: 0.3,  # Dfc
        44: 0.3,  # Dfd
        45: 0.4,  # Dsa
        46: 0.4,  # Dsb
        47: 0.4,  # Dsc
        48: 0.4,  # Dsd
        49: 0.3,  # Dwa
        50: 0.2,  # Dwb
        51: 0.1,  # Dwc
        52: 0.1,  # Dwd
        61: 0.1,  # EF
        62: 0.1   # ET
    }
    
    aridity_map = {
        11: 0.1,  # Af
        12: 0.2,  # Am
        13: 0.2,  # As
        14: 0.3,  # Aw
        21: 0.9,  # BWk
        22: 0.8,  # BWh
        26: 0.6,  # BSk
        27: 0.5,  # BSh
        31: 0.5,  # Cfa
        32: 0.5,  # Cfb
        33: 0.4,  # Cfc
        34: 0.4,  # Csa
        35: 0.5,  # Csb
        36: 0.4,  # Csc
        37: 0.3,  # Cwa
        38: 0.2,  # Cwb
        39: 0.2,  # Cwc
        41: 0.3,  # Dfa
        42: 0.4,  # Dfb
        43: 0.4,  # Dfc
        44: 0.3,  # Dfd
        45: 0.4,  # Dsa
        46: 0.4,  # Dsb
        47: 0.4,  # Dsc
        48: 0.4,  # Dsd
        49: 0.3,  # Dwa
        50: 0.3,  # Dwb
        51: 0.1,  # Dwc
        52: 0.1,  # Dwd
        61: 0.1,  # EF
        62: 0.1   # ET
    }
    
    # Convert the code into continuous values using the mappings above
    T = temperature_map.get(koppen_code, np.nan)
    P = precipitation_map.get(koppen_code, np.nan)
    S = seasonality_map.get(koppen_code, np.nan)
    A = aridity_map.get(koppen_code, np.nan)
    
    return {
        'Temperature': T,
        'Precipitation': P,
        'Seasonality': S,
        'Aridity': A
    }

def calculate_average_continuous(climate_codes):
    temperature_list = []
    precipitation_list = []
    seasonality_list = []
    aridity_list = []
    
    for code in climate_codes:
        continuous_values = climate_to_continuous(code)
        
        if not np.isnan(continuous_values['Temperature']):
            temperature_list.append(continuous_values['Temperature'])
        if not np.isnan(continuous_values['Precipitation']):
            precipitation_list.append(continuous_values['Precipitation'])
        if not np.isnan(continuous_values['Seasonality']):
            seasonality_list.append(continuous_values['Seasonality'])
        if not np.isnan(continuous_values['Aridity']):
            aridity_list.append(continuous_values['Aridity'])
    
    avg_temperature = np.nan if not temperature_list else sum(temperature_list) / len(temperature_list)
    avg_precipitation = np.nan if not precipitation_list else sum(precipitation_list) / len(precipitation_list)
    avg_seasonality = np.nan if not seasonality_list else sum(seasonality_list) / len(seasonality_list)
    avg_aridity = np.nan if not aridity_list else sum(aridity_list) / len(aridity_list)
    
    return {
        'Avg_Temperature': avg_temperature,
        'Avg_Precipitation': avg_precipitation,
        'Avg_Seasonality': avg_seasonality,
        'Avg_Aridity': avg_aridity
    }

df['climate'] = df['climate'].apply(lambda x: list(map(int, ast.literal_eval(x.replace(' ', ',')))))
df['Continuous_Climate'] = df['climate'].apply(calculate_average_continuous)

df_expanded = pd.concat([df['country'], df['Continuous_Climate'].apply(pd.Series)], axis=1)

df_expanded

In [7]:
df_expanded.to_csv('climate_in_continuous.csv')

In [8]:
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point
import random
import ast
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns


de = pd.read_csv('climate_in_continuous.csv')

In [9]:
de

In [10]:
dd = de.iloc[:, 2:]


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.cluster import KMeans

# Assuming dd is your DataFrame
# Fit KMeans
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto")
kmeans.fit(dd)

# Add labels to DataFrame
dd['Cluster'] = kmeans.labels_

# Set up the figure for 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Average Temperature vs. Average Aridity
sns.scatterplot(ax=axes[0], data=dd, x='Avg_Temperature', y='Avg_Aridity', 
                hue='Cluster', palette='viridis', s=100)
axes[0].set_title('Average Temperature vs. Average Aridity')
axes[0].set_xlabel('Average Temperature')
axes[0].set_ylabel('Average Aridity')

# Plot 2: Average Precipitation vs. Average Seasonality
sns.scatterplot(ax=axes[1], data=dd, x='Avg_Precipitation', y='Avg_Seasonality', 
                hue='Cluster', palette='viridis', s=100)
axes[1].set_title('Average Precipitation vs. Average Seasonality')
axes[1].set_xlabel('Average Precipitation')
axes[1].set_ylabel('Average Seasonality')

plt.tight_layout()
plt.show()


In [12]:
dd

In [13]:
dd['country'] = de['country']
dd[(dd['Cluster'] == 1) & (dd['Avg_Temperature'] < 0.4)]

In [14]:
dd[dd['Avg_Temperature'] > 0.9]